In [7]:
!pip install pyspark transformers sentence-transformers


In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.model_selection import train_test_split

In [2]:
# Initialize Spark Session
spark = SparkSession.builder.appName("MLBinaryClassifier").getOrCreate()

# Load all train-X.csv files
train_files = [f"../data/train-{i}.csv" for i in range(1, 9)]
df = spark.read.csv(train_files, header=True, inferSchema=True)

# Load additional data
directing_df = spark.read.json("../data/directing.json")
writing_df = spark.read.json("../data/writing.json")

# Merge additional information if a common key exists
if 'movie' in df.columns:
    df = df.join(directing_df, 'movie', 'left')
    df = df.join(writing_df, 'movie', 'left')

# Handle categorical data
categorical_cols = [col_name for col_name, dtype in df.dtypes if dtype == "string"]
for col_name in categorical_cols:
    indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_index").setHandleInvalid("keep")
    df = indexer.fit(df).transform(df)
    df = df.drop(col_name)  # Drop original categorical columns

# Convert label column to numeric
df = df.withColumn("label", col("label").cast("double"))

# Handle missing values
df = df.fillna(0)

# Define features assembler
feature_cols = [col_name for col_name in df.columns if col_name != "label"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

# Standardize numeric features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=False)
scaler_model = scaler.fit(df)
df = scaler_model.transform(df)

# Split data
train_df, val_df = df.randomSplit([0.8, 0.2], seed=42)

# Train a model
rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="label", numTrees=100, seed=42)
model = rf.fit(train_df)

# Evaluate the model
y_pred = model.transform(val_df)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(y_pred)

print("Model Accuracy:", accuracy)


Model Accuracy: 0.7066326530612245


In [6]:
df.show(20)

Py4JJavaError: An error occurred while calling o600.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 45.0 failed 1 times, most recent failure: Lost task 0.0 in stage 45.0 (TID 205) (047-CSC6Q73 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun$mapPart

In [7]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- numVotes: double (nullable = false)
 |-- label: double (nullable = false)
 |-- tconst_index: double (nullable = false)
 |-- primaryTitle_index: double (nullable = false)
 |-- originalTitle_index: double (nullable = false)
 |-- startYear_index: double (nullable = false)
 |-- endYear_index: double (nullable = false)
 |-- runtimeMinutes_index: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)



In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load MiniLM model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Function to clean and standardize text using MiniLM
def clean_text(text):
    if text is None or text.strip() == "":
        return "Unknown"  # Replace empty values with a placeholder
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return text.strip().title()  # Standardize capitalization

# Register function as a PySpark UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

clean_text_udf = udf(clean_text, StringType())

# Apply LLM-based text cleaning to movie titles
df = df.withColumn("primaryTitle_clean", clean_text_udf(col("primaryTitle_index")))
df = df.withColumn("originalTitle_clean", clean_text_udf(col("originalTitle_index")))

# Drop original columns
df = df.drop("primaryTitle_index", "originalTitle_index")


c:\Users\n.jintaganon\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df.printSchema()
df.show(10)

root
 |-- _c0: integer (nullable = true)
 |-- numVotes: double (nullable = false)
 |-- label: double (nullable = false)
 |-- tconst_index: double (nullable = false)
 |-- startYear_index: double (nullable = false)
 |-- endYear_index: double (nullable = false)
 |-- runtimeMinutes_index: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)
 |-- primaryTitle_clean: string (nullable = true)
 |-- originalTitle_clean: string (nullable = true)



Py4JJavaError: An error occurred while calling o600.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 44.0 failed 1 times, most recent failure: Lost task 0.0 in stage 44.0 (TID 204) (047-CSC6Q73 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun$mapPart

In [1]:
from pyspark.sql.functions import avg

# Compute average runtime per year
avg_runtime = df.groupBy("startYear_index").agg(avg("runtimeMinutes_index").alias("avg_runtime"))

# Fill missing runtime using the average for that year
df = df.join(avg_runtime, "startYear_index", "left")
df = df.withColumn("runtimeMinutes_index",
                   when(col("runtimeMinutes_index").isNull(), col("avg_runtime"))
                   .otherwise(col("runtimeMinutes_index")))
df = df.drop("avg_runtime")


NameError: name 'df' is not defined